# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 1 2022 2:01PM,245545,16,TASA-50012903,TASA USA Inc.,Released
1,Sep 1 2022 1:30PM,245543,19,ADV80005413,"AdvaGen Pharma, Ltd",Executing
2,Sep 1 2022 1:30PM,245543,19,ADV80005414,"AdvaGen Pharma, Ltd",Executing
3,Sep 1 2022 1:30PM,245543,19,ADV80005415,"AdvaGen Pharma, Ltd",Executing
4,Sep 1 2022 1:30PM,245543,19,ADV80005416,"AdvaGen Pharma, Ltd",Executing
5,Sep 1 2022 1:30PM,245543,19,ADV80005417,"AdvaGen Pharma, Ltd",Executing
6,Sep 1 2022 1:30PM,245543,19,ADV80005418,"AdvaGen Pharma, Ltd",Executing
7,Sep 1 2022 1:02PM,245540,10,8471745,Yusen – 3D Matrix,Released
8,Sep 1 2022 12:56PM,245539,10,MSP213632,"Methapharm, Inc.",Released
9,Sep 1 2022 12:56PM,245539,10,MSP213633,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,245537,Released,11
18,245539,Released,12
19,245540,Released,1
20,245543,Executing,6
21,245545,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245537,NaN,NaN,11.0
245539,NaN,NaN,12.0
245540,NaN,NaN,1.0
245543,NaN,6.0,NaN
245545,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,0.0,1.0,9.0
245470,0.0,0.0,1.0
245478,9.0,17.0,15.0
245500,0.0,0.0,1.0
245512,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,0,1,9
245470,0,0,1
245478,9,17,15
245500,0,0,1
245512,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,0,1,9
1,245470,0,0,1
2,245478,9,17,15
3,245500,0,0,1
4,245512,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,,1,9
1,245470,,,1
2,245478,9,17,15
3,245500,,,1
4,245512,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 1 2022 2:01PM,245545,16,TASA USA Inc.
1,Sep 1 2022 1:30PM,245543,19,"AdvaGen Pharma, Ltd"
7,Sep 1 2022 1:02PM,245540,10,Yusen – 3D Matrix
8,Sep 1 2022 12:56PM,245539,10,"Methapharm, Inc."
20,Sep 1 2022 12:52PM,245537,10,ISDIN Corporation
31,Sep 1 2022 12:49PM,245530,10,ISDIN Corporation
87,Sep 1 2022 12:46PM,245535,10,"Snap Medical Industries, LLC"
88,Sep 1 2022 12:44PM,245534,10,Emerginnova
91,Sep 1 2022 12:40PM,245533,12,Hush Hush
92,Sep 1 2022 12:17PM,245529,10,"Lupin Research, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 1 2022 2:01PM,245545,16,TASA USA Inc.,,,1
1,Sep 1 2022 1:30PM,245543,19,"AdvaGen Pharma, Ltd",,6,
2,Sep 1 2022 1:02PM,245540,10,Yusen – 3D Matrix,,,1
3,Sep 1 2022 12:56PM,245539,10,"Methapharm, Inc.",,,12
4,Sep 1 2022 12:52PM,245537,10,ISDIN Corporation,,,11
5,Sep 1 2022 12:49PM,245530,10,ISDIN Corporation,,,56
6,Sep 1 2022 12:46PM,245535,10,"Snap Medical Industries, LLC",,,1
7,Sep 1 2022 12:44PM,245534,10,Emerginnova,,,3
8,Sep 1 2022 12:40PM,245533,12,Hush Hush,,1,
9,Sep 1 2022 12:17PM,245529,10,"Lupin Research, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 2:01PM,245545,16,TASA USA Inc.,1,,
1,Sep 1 2022 1:30PM,245543,19,"AdvaGen Pharma, Ltd",,6,
2,Sep 1 2022 1:02PM,245540,10,Yusen – 3D Matrix,1,,
3,Sep 1 2022 12:56PM,245539,10,"Methapharm, Inc.",12,,
4,Sep 1 2022 12:52PM,245537,10,ISDIN Corporation,11,,
5,Sep 1 2022 12:49PM,245530,10,ISDIN Corporation,56,,
6,Sep 1 2022 12:46PM,245535,10,"Snap Medical Industries, LLC",1,,
7,Sep 1 2022 12:44PM,245534,10,Emerginnova,3,,
8,Sep 1 2022 12:40PM,245533,12,Hush Hush,,1,
9,Sep 1 2022 12:17PM,245529,10,"Lupin Research, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 2:01PM,245545,16,TASA USA Inc.,1,,
1,Sep 1 2022 1:30PM,245543,19,"AdvaGen Pharma, Ltd",,6,
2,Sep 1 2022 1:02PM,245540,10,Yusen – 3D Matrix,1,,
3,Sep 1 2022 12:56PM,245539,10,"Methapharm, Inc.",12,,
4,Sep 1 2022 12:52PM,245537,10,ISDIN Corporation,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 2:01PM,245545,16,TASA USA Inc.,1.0,NaN,NaN
1,Sep 1 2022 1:30PM,245543,19,"AdvaGen Pharma, Ltd",NaN,6.0,NaN
2,Sep 1 2022 1:02PM,245540,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Sep 1 2022 12:56PM,245539,10,"Methapharm, Inc.",12.0,NaN,NaN
4,Sep 1 2022 12:52PM,245537,10,ISDIN Corporation,11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 2:01PM,245545,16,TASA USA Inc.,1.0,0.0,0.0
1,Sep 1 2022 1:30PM,245543,19,"AdvaGen Pharma, Ltd",0.0,6.0,0.0
2,Sep 1 2022 1:02PM,245540,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Sep 1 2022 12:56PM,245539,10,"Methapharm, Inc.",12.0,0.0,0.0
4,Sep 1 2022 12:52PM,245537,10,ISDIN Corporation,11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3191793,91.0,0.0,0.0
12,245533,0.0,1.0,0.0
15,245478,15.0,17.0,9.0
16,245545,1.0,0.0,0.0
19,245543,0.0,6.0,0.0
20,245453,9.0,1.0,0.0
21,245520,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3191793,91.0,0.0,0.0
1,12,245533,0.0,1.0,0.0
2,15,245478,15.0,17.0,9.0
3,16,245545,1.0,0.0,0.0
4,19,245543,0.0,6.0,0.0
5,20,245453,9.0,1.0,0.0
6,21,245520,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,91.0,0.0,0.0
1,12,0.0,1.0,0.0
2,15,15.0,17.0,9.0
3,16,1.0,0.0,0.0
4,19,0.0,6.0,0.0
5,20,9.0,1.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,91.0
1,12,Released,0.0
2,15,Released,15.0
3,16,Released,1.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,9.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,17.0,0.0,6.0,1.0,1.0
Released,91.0,0.0,15.0,1.0,0.0,9.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,9.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,17.0,0.0,6.0,1.0,1.0
2,Released,91.0,0.0,15.0,1.0,0.0,9.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,9.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,17.0,0.0,6.0,1.0,1.0
2,Released,91.0,0.0,15.0,1.0,0.0,9.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()